In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from secret import username, password
from sqlalchemy import create_engine

In [2]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'

### Mac (Irina) Connection

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Windows (Clay) Connection

In [1]:
# executable_path = {'executable_path': 'C:/Users/Clay/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [5]:
browser.visit(url)

In [6]:
tables = pd.read_html(url)

In [7]:
# tables

In [8]:
table_df = tables[0]

In [9]:
table_df.set_index('Rank')
table_df.head()

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]


In [10]:
table_df.drop(columns=['Peak', 'Year', 'Reference(s)'], inplace=True)

In [12]:
table_df = table_df.reset_index()

In [13]:
ww_gross_series = table_df['Worldwide gross']

In [14]:
len(ww_gross_series)

50

In [16]:
type(ww_gross_series)

pandas.core.series.Series

In [17]:
list = []
for i in ww_gross_series:
    print(i.split('$')[-1])
    list.append(i.split('$')[-1]) 

2,797,800,564
2,790,439,000
2,194,439,542
2,068,223,624
2,048,359,754
1,671,713,208
1,656,943,394
1,518,812,988
1,516,045,911
1,450,026,933
1,402,805,868
1,346,913,161
1,341,932,398
1,332,539,889
1,309,484,461
1,290,000,000
1,263,521,126
1,242,805,359
1,238,764,765
1,214,811,252
1,159,398,397
1,153,304,495
1,148,161,807
1,142,219,401
1,131,927,996
1,128,274,794
1,123,794,079
1,108,561,013
1,104,054,072
1,084,939,099
1,074,251,311
1,074,144,248
1,073,394,593
1,066,969,703
1,066,179,725
1,056,057,273
1,050,693,953
1,045,713,802
1,034,799,409
1,029,939,903
1,028,570,889
1,027,044,677
1,025,467,110
1,023,784,195
1,021,103,568
1,004,934,033
978,087,613
976,920,103
970,761,885
968,483,777


In [18]:
split_list = []
for i in list:
    i = i.split(',')
    split_list.append(''.join(i)) 

In [19]:
merged_list = []
for i in split_list:
    print(''.join(i))
    merged_list.append(''.join(i)) 

2797800564
2790439000
2194439542
2068223624
2048359754
1671713208
1656943394
1518812988
1516045911
1450026933
1402805868
1346913161
1341932398
1332539889
1309484461
1290000000
1263521126
1242805359
1238764765
1214811252
1159398397
1153304495
1148161807
1142219401
1131927996
1128274794
1123794079
1108561013
1104054072
1084939099
1074251311
1074144248
1073394593
1066969703
1066179725
1056057273
1050693953
1045713802
1034799409
1029939903
1028570889
1027044677
1025467110
1023784195
1021103568
1004934033
978087613
976920103
970761885
968483777


In [20]:
len(merged_list)

50

In [21]:
merged_list

['2797800564',
 '2790439000',
 '2194439542',
 '2068223624',
 '2048359754',
 '1671713208',
 '1656943394',
 '1518812988',
 '1516045911',
 '1450026933',
 '1402805868',
 '1346913161',
 '1341932398',
 '1332539889',
 '1309484461',
 '1290000000',
 '1263521126',
 '1242805359',
 '1238764765',
 '1214811252',
 '1159398397',
 '1153304495',
 '1148161807',
 '1142219401',
 '1131927996',
 '1128274794',
 '1123794079',
 '1108561013',
 '1104054072',
 '1084939099',
 '1074251311',
 '1074144248',
 '1073394593',
 '1066969703',
 '1066179725',
 '1056057273',
 '1050693953',
 '1045713802',
 '1034799409',
 '1029939903',
 '1028570889',
 '1027044677',
 '1025467110',
 '1023784195',
 '1021103568',
 '1004934033',
 '978087613',
 '976920103',
 '970761885',
 '968483777']

In [22]:
# Rename the columns to be able to merge the data 
table_df.rename(columns={"Title": "name", "Rank": "rank", "Worldwide gross":"worldwide gross"}, inplace=True)

In [23]:
# Change revenue column from string to big integer
table_df['ww_gross'] = merged_list

In [24]:
table_df.head()

,index,rank,name,worldwide gross,ww_gross
0,0,1,Avengers: Endgame,"$2,797,800,564",2797800564
1,1,2,Avatar,"$2,790,439,000",2790439000
2,2,3,Titanic,"$2,194,439,542",2194439542
3,3,4,Star Wars: The Force Awakens,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,"$2,048,359,754",2048359754


### Connect to local database

In [25]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [26]:
engine.table_names()

['domestic_revenue', 'international']

### Use pandas to load csv converted DataFrame into database

In [27]:
table_df.to_sql(name='international', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the international table

In [28]:
pd.read_sql_query('select * from international', con=engine).head()

,index,rank,name,worldwide gross,ww_gross
0,0,1,Avengers: Endgame,"$2,797,800,564",2797800564
1,1,2,Avatar,"$2,790,439,000",2790439000
2,2,3,Titanic,"$2,194,439,542",2194439542
3,3,4,Star Wars: The Force Awakens,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,"$2,048,359,754",2048359754


In [47]:
pd.read_sql_query('select * from domestic_revenue', con=engine).head()

,id,gross,name
0,229,52287414,Stand by Me
1,230,70136369,Ferris Bueller's Day Off
2,231,179800601,Top Gun
3,232,85160248,Aliens
4,233,18564613,Flight of the Navigator


### Check The-Numbers website for international data

In [2]:
import time

In [13]:
url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time'

In [ ]:
# Create for loop to iterate over the pages


In [68]:
list = []
pages_num = 3
base_url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time'
page_num = 101

for i in range(pages_num):
    print(i)
    if page_num == 101:
        page_url = (f"{base_url}/{str(page_url)}")
        page_num = page_num
        print(page_url)
    else:
        page_num = 
print(page_num)      

0
https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/101
1
https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/101
2
https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time/101
101


In [25]:
list_test = []
base_url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time'
page_num = 1
table_list = []

for i in range(1, 301, 100):
    if page_num < 100:
        print("First FOR branch < 100")
        url=(f"{base_url}")
        print(url)
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(url)
        print("sleep 5")
        time.sleep(5)
        tables = pd.read_html(url)
        time.sleep(5)
        browser.quit()
        page_num += 100
#     else:
#         print(i)
#         url=(f"{base_url}/{str(i)}")
#         print(url)
#         executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#         browser = Browser('chrome', **executable_path, headless=False)
#         browser.visit(url)
#         time.sleep(3)
#         tables = pd.read_html(url)
#         time.sleep(3)
#         page_num += 100
#         browser.quit()

    

First FOR branch < 100
https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time
sleep 5


HTTPError: HTTP Error 403: Forbidden

In [23]:
browser.quit()

### Best movie directors of all times

In [26]:
movie_dir_url = 'https://www.imdb.com/list/ls056848274/'

In [27]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
browser.visit(movie_dir_url)
time.sleep(5)

In [18]:
tables = pd.read_html(url)

HTTPError: HTTP Error 403: Forbidden

In [19]:
# time.sleep(5)

In [20]:
# browser.quit()
# page_num += 100

In [ ]:
print()

In [53]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [54]:
browser.visit(numbers_url)

In [55]:
tables = pd.read_html(url)

In [56]:
numbers_table_df = tables[0]

In [60]:
numbers_table_df.head()

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]


In [59]:
browser.quit()

### Check The-Numbers website for international data

In [48]:
numbers_url = 'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time'

In [53]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [54]:
browser.visit(numbers_url)

In [55]:
tables = pd.read_html(url)

In [56]:
numbers_table_df = tables[0]

In [58]:
numbers_table_df

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]
5,6,3,Jurassic World,"$1,671,713,208",2015,[# 11][# 12]
6,7,7,The Lion King,"$1,656,943,394",2019,[# 13][# 2]
7,8,3,The Avengers,"$1,518,812,988",2012,[# 14][# 15]
8,9,4,Furious 7,"$1,516,045,911",2015,[# 16][# 17]
9,10,10,Frozen II,"$1,450,026,933",2019,[# 18]


In [59]:
browser.quit()

### Box Office Mojo

In [29]:
mojo_url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW'

In [30]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
browser.visit(mojo_url)

In [32]:
tables = pd.read_html(mojo_url)

In [33]:
numbers_table_df = tables[0]

In [34]:
numbers_table_df

,Rank,Title,Lifetime Gross,Year
0,1,Avengers: Endgame,"$2,797,800,564",2019
1,2,Avatar,"$2,790,439,000",2009
2,3,Titanic,"$2,195,169,138",1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624",2015
4,5,Avengers: Infinity War,"$2,048,359,754",2018
...,...,...,...,...
195,196,The Revenant,"$532,950,503",2015
196,197,The Meg,"$530,254,422",2016
197,198,Ralph Breaks the Internet,"$529,323,962",2018
198,199,Hotel Transylvania 3: Summer Vacation,"$528,583,774",2018


In [35]:
mojo_url_2 = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=200'

In [30]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
browser.visit(mojo_url)

In [32]:
tables = pd.read_html(mojo_url)

In [33]:
numbers_table_df = tables[0]

In [34]:
numbers_table_df

,Rank,Title,Lifetime Gross,Year
0,1,Avengers: Endgame,"$2,797,800,564",2019
1,2,Avatar,"$2,790,439,000",2009
2,3,Titanic,"$2,195,169,138",1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624",2015
4,5,Avengers: Infinity War,"$2,048,359,754",2018
...,...,...,...,...
195,196,The Revenant,"$532,950,503",2015
196,197,The Meg,"$530,254,422",2016
197,198,Ralph Breaks the Internet,"$529,323,962",2018
198,199,Hotel Transylvania 3: Summer Vacation,"$528,583,774",2018


In [3]:
mojo_list = []
base_url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW'
page_num = 0
table_mojo_df = []
table_on_page = ""

for i in range(0, 10200, 200):
    if page_num < 100:
        print("Printing first 200 rows.")
        url=(f"{base_url}")
        print(url)
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(url)
        time.sleep(5)
        print("sleep 5")
        tables = pd.read_html(url)
        time.sleep(5)
        print("sleep 5")
        table_mojo_df = tables[0]
        browser.quit()
        page_num += 200
        table_mojo_df = table_mojo_df.append(table_mojo_df)
        table_page = (f"Table_{str(i)}")
        print(table_page)
        print(type(table_page))
        table_page = table_mojo_df
        print(table_page)
        table_mojo_df.to_csv(f"csv/Table_{str(i)}.csv")
        print('-------------------------------------')
    else:
        print(f"Printing pages {i+1} to {i+200}")
        url=(f"{base_url}&offset={str(i)}")
        print(url)
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(url)
        time.sleep(5)
        print("sleep 5")
        tables = pd.read_html(url)
        time.sleep(5)
        print("sleep 5")
        table_mojo_df = tables[0]
        browser.quit()
        page_num += 200
        table_mojo_df = table_mojo_df.append(table_mojo_df)
#         table_page = str(i)
        table_page = (f"Table_{str(i)}")
        print(table_page)
        print(type(table_page))
        table_page = table_mojo_df
        print(table_page)
        table_mojo_df.to_csv(f"csv/Table_{str(i)}.csv")
        print('-------------------------------------')

    

Printing first 200 rows.
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW
sleep 5
sleep 5
Table_0
<class 'str'>
     Rank                                       Title  Lifetime Gross  Year
0       1                           Avengers: Endgame  $2,797,800,564  2019
1       2                                      Avatar  $2,790,439,000  2009
2       3                                     Titanic  $2,195,169,138  1997
3       4  Star Wars: Episode VII - The Force Awakens  $2,068,223,624  2015
4       5                      Avengers: Infinity War  $2,048,359,754  2018
..    ...                                         ...             ...   ...
195   196                                The Revenant    $532,950,503  2015
196   197                                     The Meg    $530,254,422  2016
197   198                   Ralph Breaks the Internet    $529,323,962  2018
198   199       Hotel Transylvania 3: Summer Vacation    $528,583,774  2018
199   200                           

sleep 5
sleep 5
Table_1800
<class 'str'>
     Rank                Title Lifetime Gross  Year
0    1801  Season of the Witch    $91,627,228  2011
1    1802                Crawl    $91,542,097  2019
2    1803      Five Feet Apart    $91,522,094  2019
3    1804      Friday the 13th    $91,509,154  2009
4    1805    Rambo: Last Blood    $91,490,353  2019
..    ...                  ...            ...   ...
195  1996    Go Away Mr. Tumor    $80,606,893  2015
196  1997        Vampires Suck    $80,547,866  2010
197  1998       Den of Thieves    $80,509,622  2018
198  1999            Halloween    $80,460,948  2007
199  2000  Muppets Most Wanted    $80,383,113  2014

[400 rows x 4 columns]
-------------------------------------
Printing pages 2001 to 2200
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=2000
sleep 5
sleep 5
Table_2000
<class 'str'>
     Rank                                     Title Lifetime Gross  Year
0    2001                                    Takers   

sleep 5
sleep 5
Table_3600
<class 'str'>
     Rank                                   Title Lifetime Gross  Year
0    3601    The Adventures of Rocky & Bullwinkle    $35,134,820  2000
1    3602                             Miss Potter    $35,078,241  2006
2    3603                                   Alfie    $35,060,882  2004
3    3604                    My Week with Marilyn    $35,057,696  2011
4    3605                          The Liquidator    $35,046,130  2017
..    ...                                     ...            ...   ...
195  3796  The House That Never Dies: Reawakening    $32,433,588  2017
196  3797                             Black Sheep    $32,417,995  1996
197  3798                  The Invention of Lying    $32,406,507  2009
198  3799                                 Dhoom 2    $32,396,427  2006
199  3800                     Rumble in the Bronx    $32,392,047  1996

[400 rows x 4 columns]
-------------------------------------
Printing pages 3801 to 4000
https://www.boxof

sleep 5
sleep 5
Table_5400
<class 'str'>
     Rank                         Title Lifetime Gross  Year
0    5401                From Me to You    $18,033,727  2010
1    5402  Mr. Pride vs. Miss Prejudice    $18,019,651  2017
2    5403                    Cape No. 7    $18,015,600  2008
3    5404             The Country Bears    $18,012,097  2002
4    5405         Harlock: Space Pirate    $18,010,317  2013
..    ...                           ...            ...   ...
195  5596      Kung Pow: Enter the Fist    $16,994,625  2002
196  5597                Death Sentence    $16,974,459  2007
197  5598                Chinese Puzzle    $16,968,297  2013
198  5599          Till Luck Do Us Part    $16,963,608  2012
199  5600                Catch That Kid    $16,951,702  2004

[400 rows x 4 columns]
-------------------------------------
Printing pages 5601 to 5800
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=5600
sleep 5
sleep 5
Table_5600
<class 'str'>
     Rank          

sleep 5
sleep 5
Table_7200
<class 'str'>
     Rank                                           Title Lifetime Gross  Year
0    7201                                  Thread of Lies    $10,522,797  2014
1    7202                                         Get Low    $10,522,511  2010
2    7203                            Black Widow Business    $10,504,345  2016
3    7204  Até Que a Sorte nos Separe 3: A Falência Final    $10,502,937  2015
4    7205                                        Anatomie    $10,501,958  2000
..    ...                                             ...            ...   ...
195  7396                                    The Deep End    $10,031,529  2001
196  7397                             Tear This Heart Out    $10,031,226  2008
197  7398                Kisarazu Cat's Eye: Nihon Series    $10,029,866  2003
198  7399                                    Agathe Cléry    $10,025,910  2008
199  7400                                     Crying Fist    $10,024,751  2005

[400 rows 

sleep 5
sleep 5
Table_8800
<class 'str'>
     Rank                                              Title Lifetime Gross  \
0    8801                                               Goon     $6,985,158   
1    8802                                      Psychokinesis     $6,981,970   
2    8803       Children of the Corn II: The Final Sacrifice     $6,980,986   
3    8804                                       Donnie Darko     $6,979,093   
4    8805                                Kôdaike no hitobito     $6,978,689   
..    ...                                                ...            ...   
195  8996                                   Forced Vengeance     $6,660,333   
196  8997                             Where the Buffalo Roam     $6,659,377   
197  8998                                        Son of Saul     $6,659,121   
198  8999  InuYasha the Movie 3: Swords of an Honorable R...     $6,654,841   
199  9000                                        April Fools     $6,652,777   

     Year 

In [47]:
type(table_page)

pandas.core.frame.DataFrame

In [53]:
table_page.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [53]:
table_page.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [54]:
table_mojo_df.head()

,Rank,Title,Lifetime Gross,Year
0,601,Us,"$255,184,580",2019
1,602,Cliffhanger,"$255,000,211",1993
2,603,Baahubali 2: The Conclusion,"$254,158,390",2017
3,604,Babe,"$254,134,910",1995
4,605,Men in Black: International,"$253,890,701",2019


In [55]:
table_mojo_df.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [58]:
test = str(400)

In [59]:
test.head()

AttributeError: 'str' object has no attribute 'head'